# Workshop I - Systems Analysis: Entrophy

## Workshop Definition:

Welcome to the first workshop of Systems Analysis course. Let’s funny me with a_ bioinformatics_ exercise.

Imagine you have been hired as __data analyst__ in an important biotechnology company.  Your boss, a Science Chief Officer, wants to get some _patterns_ in genomic data, sometimes called __motifs__.

Here you will have some tasks in order to complete this _workshop_:

1. Create a _dummy database_ of genetic sequences composed of nucleotide bases (_A_, _C_, _G_, _T_), where each sequence must have between $10$ and $20$ bases. Your database must be composed for $50.000$ genetic sequences.
1. Get the _motifs_ (must repeated sequence) of size $6$ and $8$.
1. Use the __Shannon Entropy__ measurement to filter sequences with not a good variance level.
1. Get again the _motifs_ of size $6$ and $8$.

Get some conclusions based on your analysis.

Write any technical concern/decision/difficulty  you think is relevant regarding your work.
You must deliver a full report detailing each one of the previous steps. For steps 1 to 4 you must describe the algorithms you propose and let an screenshot about the code and the output of the code. I strongly recommend you to use a _Jupyter Notebook_ or a _COLAB_ to write/execute your code.

1. First, a function to generate a sequence based on typical for DNA nucleotid bases is created. Here the idea is to use random numbers to get a random sequence of a random size (between 10 and 20.)

In [1]:
import random


def create_sequence() -> str:
    """
    This function is used to generate a random genetic sequence.

    Returns:
    - str: random genetic sequence
    """
    nucleotid_bases = ["A", "C", "G", "T"]
    size_sequence = random.randint(10, 20)
    new_sequence = [nucleotid_bases[random.randint(0, 3)] for i in range(size_sequence)]
    return "".join(new_sequence)

2. Based on `create_sequences()` function, the idea is to generate a dataset of genetic sequences, just persisting all sequences into a list.

In [2]:
def create_dataset(dataset_size: int) -> list:
    """
    This function is used to create a dataset composed by a set of genetic sequences.

    Parameters:
    - dataset_size (int): size of the dummy dataset to be generated.

    Returns:
    - list: a list of genetic sequences
    """
    dataset = [create_sequence() for i in range(dataset_size)]
    return dataset

3. As the goal is to get the motif in the sequence, first task for that purpose is to generate all possible combinations of secuences of an specific size. 

It means, in `size = 3`, the valid combinations would be: `[AAA, AAC, AAG, AAT, CAA, CAC, CAC,..., TTG, TTT]` 

In [3]:
def get_combinations(n: int, sequences: list, bases: list) -> list:
    """
    This method is used to generate a set of combinations based on a list of nucleotid bases.
    To make easy the process, this function is defined as a recurssion.

    Parameters:
    - n (int): amount of elements of each combination
    - sequences (list): list of recursive sequences obtained
    - bases (list): list of nucleotid bases to be used

    Returns:
    - list: list of combinations
    """
    if n == 1:
        return [sequence + base for sequence in sequences for base in bases]
    else:
        sequence_ = [sequence + base for sequence in sequences for base in bases]
        return get_combinations(n - 1, sequence_, bases)

4. Each motif candidate should be verified into each sequence in the dataset, just to count the motif occurences.

In [4]:
def count_motif(motif: str, sequences_dataset: list) -> int:
    """
    This function is used to count the number of times a motif appears in a set of genetic sequences.

    Parameters:
    - motif (str): genetic motif to be searched.
    - sequences_dataset (list): list of genetic sequences.

    Returns:
    - int: number of times the motif appears in the dataset.
    """
    count = 0
    for sequence in sequences_dataset:
        count += sequence.count(motif)
    return count

5. Now it is time to get all motif candidates (combinations of a specific size), count occurrences for each motif, and choose as the motif winner the only one who has more ocurreces.

In [5]:
def get_motif(motif_size: int, sequences_dataset: list) -> (str, int):
    """
    This function is used to get the motif with the highest count in a set of genetic sequences.

    Parameters:
    - motif_size (int): size of the motif to be searched.
    - sequences_db (list): list of genetic sequences.

    Returns:
    - (str, int): motif with the highest count and the number of times it appears in the dataset.
    """
    nucleotid_bases = ["A", "C", "G", "T"]
    combinations = get_combinations(motif_size, [""], nucleotid_bases)
    # get motif with the highest count
    max_counter = 0
    motif_winner = ""
    for motif_candidate in combinations:
        temp_counter = count_motif(motif_candidate, sequences_dataset)
        if temp_counter > max_counter:
            max_counter = temp_counter
            motif_winner = motif_candidate

    return motif_winner, max_counter

6. Now, just a loop of callings looking for some different motifs, in order you have a better idea about possible motifs and occurences, so you could make a better analysis. Also, motifs of size 6 and 8 will be generated.

In [6]:
for size in [6, 8]:
    print(f"\nMotifs of size: {size}")
    for i in range(10):
        print(get_motif(size, create_dataset(50000)))


Motifs of size: 6
('TGCGCC', 163)
('CCGGAG', 169)
('GAGAGT', 159)
('ATGGTG', 162)
('CGTGAG', 173)
('GACTTG', 164)
('CTAACT', 164)
('TAGGTT', 163)
('GGATTC', 163)
('TTGAAC', 163)

Motifs of size: 8
('AATGTCAA', 20)
('ACAAGCAA', 19)
('TGTTCTCA', 19)
('TTGATGGC', 21)
('CCGGCCTT', 18)
('ATTGACGG', 18)
('ACACTCTC', 21)
('ATTGGATG', 21)
('CCACGCCC', 19)
('CATGGGAT', 19)


7. Now, as the idea is to define how Shannon Entropy could help us to filter maybe not relevant sequences in order to get more relevant motifs. You could check equation [here](https://rosettacode.org/wiki/Entropy).

In [7]:
import math


def calculate_shannon_entrophy(sequence: str) -> float:
    """
    This function is used to calculate the Shannon Entropy of a genetic sequence.

    Parameters:
    - sequence (str): genetic sequence.

    Returns:
    - float: Shannon Entropy of the sequence.
    """
    size_sequence = len(sequence)
    bases = ["A", "C", "G", "T"]
    entrophy = 0
    for base in bases:
        count = sequence.count(base)
        if count > 0:
            p = (count / size_sequence) * math.log2(count / size_sequence)
        else:
            p = 0
        entrophy += p
    entrophy *= -1
    return entrophy

8. As a Shannon Entropy Calculation function is created, it is possible to make a dataset filter before to check for motifs.

In [8]:
def filter_shannon(sequence: str) -> bool:
    """
    This function is used to filter genetic sequences based on their Shannon Entropy.

    Parameters:
    - sequence (str): genetic sequence.

    Returns:
    - bool: True if the sequence passes the filter, False otherwise.
    """
    return calculate_shannon_entrophy(sequence) > 1.8

9. Last part of the workshop, it is just filter sequences in the full motif search process.

In [9]:
for size in [6, 8]:
    print(f"\nArter filter, motifs of size: {size}")
    for i in range(10):
        dataset = create_dataset(50000)
        dataset = list(filter(filter_shannon, dataset))
        print(f"Dataset size: {len(dataset)}, Motif: {get_motif(size, dataset)}")


Arter filter, motifs of size: 6
Dataset size: 35439, Motif: ('ACTGTC', 141)
Dataset size: 35521, Motif: ('AATCGT', 140)
Dataset size: 35578, Motif: ('CATCGA', 143)
Dataset size: 35340, Motif: ('CGAGCT', 140)
Dataset size: 35471, Motif: ('TAGCAT', 146)
Dataset size: 35502, Motif: ('TCGTGA', 142)
Dataset size: 35511, Motif: ('CCTTAG', 148)
Dataset size: 35388, Motif: ('ACAGCT', 137)
Dataset size: 35488, Motif: ('AGTCTG', 140)
Dataset size: 35424, Motif: ('TAGTAC', 141)

Arter filter, motifs of size: 8
Dataset size: 35350, Motif: ('TCTTAAAG', 18)
Dataset size: 35399, Motif: ('AGGTTTAG', 17)
Dataset size: 35499, Motif: ('ACGCATAG', 18)
Dataset size: 35310, Motif: ('CCTGGACT', 20)
Dataset size: 35548, Motif: ('GCCGCATT', 19)
Dataset size: 35408, Motif: ('GCAATGTG', 18)
Dataset size: 35431, Motif: ('TTGAATCG', 21)
Dataset size: 35407, Motif: ('TCGCATAG', 18)
Dataset size: 35469, Motif: ('CAGGTATC', 17)
Dataset size: 35412, Motif: ('GACGTTAC', 17)
